![First Power Bi](.\images\dbachecks-logo.png )
# dbachecks

## Installation 

Take a look at the installation guidelines here [Intro](./01-dbachecks-Intro.ipynb)

Lets Check if you have the dbachecks module available

In [1]:
Get-Module dbachecks -ListAvailable



    Directory: C:\Users\mrrob\Documents\PowerShell\Modules

ModuleType Version    PreRelease Name                                PSEdition ExportedCommands
---------- -------    ---------- ----                                --------- ----------------
Script     1.2.24                dbachecks                           Desk      {Get-DbcConfig, Get…
Script     1.2.20                dbachecks                           Desk      {Get-DbcConfig, Get…
Script     1.2.15                dbachecks                           Desk      {Get-DbcConfig, Get…
Script     1.2.14                dbachecks                           Desk      {Get-DbcConfig, Get…
Script     1.2.13                dbachecks                           Desk      {Get-DbcConfig, Get…
Script     1.2.12                dbachecks                           Desk      {Get-DbcConfig, Get…

    Directory: C:\Program Files\WindowsPowerShell\Modules

ModuleType Version    PreRelease Name                                PSEdition Exporte

## Examine the available commands

You can look at the commands in the dbachecks module

In [ ]:
Get-Command -Module dbachecks

# All of the checks

You can see all of the checks using `Get-DbcCheck` 


In [ ]:
Get-DbcCheck

## Searching for Checks to run

You can search for checks using `Get-DbcCheck -Pattern SEARCHTERM`

In [12]:
Get-DbcCheck -Pattern backup

# Running a Check

This requires that you have the containers set up from the Notebook 00 - Setting up the containers for the rest of the containers.ipynb

If you have altered the folder path you will need to alter it below.

Running a check from the command line can be done with

`Invoke-DbcCheck -SqlInstance SQLINSTANCE -Check UNIQUETAG, A TAG, GROUP`

You can run multiple checks with

`Invoke-DbcCheck -SqlInstance SQLINSTANCE -Check Tag,AnotherTag,AnotherTag`

In [15]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstance = 'localhost,15592'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
$null = Reset-DbcConfig
Invoke-DbcCheck -SqlInstance $SqlInstance -SqlCredential $SqlCredential -Check AutoClose

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Database.Tests.ps1' with Tags AutoClose

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Database.Tests.ps1

  Describing Auto Close

    Context Testing Auto Close on localhost,15592
      [+] AdventureWorks2017 on localhost,15592 should have Auto Close set to False 36ms
      [+] master on localhost,15592 should have Auto Close set to False 34ms
      [+] model on localhost,15592 should have Auto Close set to False 37ms
      [+] msdb on localhost,15592 should have Auto Close set to False 40ms
      [+] Northwind on localhost,15592 should have Auto Close set to False 36ms
      [+] pubs on localhost,15592 should have Auto Close set to False 39ms
      [+] tempdb on localhost,15592 should have Aut

## Auto-Complete for Checks
At the command line and in the editor the Checks will auto-complete

Place the cursor a space after the `-Check` and press CTRL + SPACE

`Invoke-DbcCheck -SqlInstance SQL2016N1 -Check `

In [4]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstance = 'localhost,15592'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstance -SqlCredential $SqlCredential -Check AdHocWorkload

Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Instance.Tests.ps1' with Tags AdHocWorkload

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Instance.Tests.ps1

  Describing Ad Hoc Workload Optimization

    Context Testing Ad Hoc Workload Optimization on localhost,15592
      [+] localhost,15592 Should be Optimize for Ad Hoc workloads 133ms
Tests completed in 656ms
Tests Passed: 1, Failed: 0, Skipped: 0, Pending: 0, Inconclusive: 0 


# Check Agent Jobs at the command line

Did all of the Agent Jobs succeed ?

In [16]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstance = 'localhost,15592'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstance -SqlCredential $SqlCredential -Check FailedJob

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Agent.Tests.ps1' with Tags FailedJob

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Agent.Tests.ps1

  Describing Failed Jobs

    Context Checking for failed enabled jobs since 02/13/2020 17:54:43 on localhost,15592
Tests completed in 822ms
Tests Passed: 0, Failed: 0, Skipped: 0, Pending: 0, Inconclusive: 0 


You may have no results because they are no enabled jobs that have run in the last 7 days (depending upon when I last updated the image). Lets run the jobs and see what we get

In [6]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstance = 'localhost,15592'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
(Get-DbaAgentJob -SqlInstance $SqlInstance -SqlCredential $SqlCredential).Start()
Start-Sleep -Seconds 15
Invoke-DbcCheck -SqlInstance $SqlInstance -SqlCredential $SqlCredential -Check FailedJob

Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Agent.Tests.ps1' with Tags FailedJob

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Agent.Tests.ps1

  Describing Failed Jobs

    Context Checking for failed enabled jobs since 02/13/2020 15:46:18 on localhost,15592
      [+] CommandLog Cleanup's last run outcome on 0034794b4ce3 is Succeeded 10ms
      [-] DatabaseBackup - SYSTEM_DATABASES - FULL's last run outcome on 0034794b4ce3 is Failed 182ms
        Expected 'Succeeded', because All Agent Jobs should have succeed - you need to investigate the failed jobs, but got Failed.
        170:                                         $psitem.LastRunOutcome | Should -Be "Succeeded" -Because 'All Agent Jobs should have succeed - you need to investigate the failed jobs'
        at <ScriptBlock>, C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Agent.Tests.ps1: line 170
      [-] DatabaseBackup - USE

## Check the instances are available
Are all of the instances up and available?

There will be failures for the default values of the authentication scheme as we are connecting with SQL Authentication. We will look at configuration in a later notebook.

In [17]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential -Check InstanceConnection 

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Instance.Tests.ps1' with Tags InstanceConnection

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Instance.Tests.ps1

  Describing Instance Connection

    Context Testing Instance Connection on localhost,15592
      [+] connects successfully to localhost,15592 5ms
      [-] auth scheme should be Kerberos on localhost,15592 116ms
        Expected strings to be the same, but they were different.
        Expected length: 8
        Actual length:   3
        Strings differ at index 0.
        Expected: 'Kerberos'
        But was:  'SQL'
        64:                         (Test-DbaConnectionAuthScheme -SqlInstance $Instance).authscheme | Should -Be $authscheme
        at <ScriptBlock>, C:\Users\mrrob\

# Check the databases are available
Are all the databases up available?

This by default will only show the system databases but we will talk about configuration for this in the next notebook

In [18]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential  -Check DatabaseStatus

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Database.Tests.ps1' with Tags DatabaseStatus

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Database.Tests.ps1

  Describing Database Status

    Context Database status is correct on localhost,15592
      [+] Database AdventureWorks2017 on localhost,15592 has the expected status 153ms
      [+] Database master on localhost,15592 has the expected status 148ms
      [+] Database model on localhost,15592 has the expected status 143ms
      [+] Database msdb on localhost,15592 has the expected status 165ms
      [+] Database Northwind on localhost,15592 has the expected status 155ms
      [+] Database pubs on localhost,15592 has the expected status 139ms
      [+] Database tempdb on localhost,15592 

## Check that DBCC CheckDB 

This is one of my favourite checks to run. You can get the last time DBCC CHECKDB was run for a single database using T-SQL with

```sql
DBCC DBINFO('DatabaseName') WITH TABLERESULTS
```

You will find the result somewhere around line 50. Doing that for all of the databases on your estate is time consuming and prone to (human) error.
With dbachecks you can do it like this. ON my machine this checks the 2 containers and 14 databases to make sure DBCC CHECKDB was run in the last 7 days in under 2 seconds :-)

In [19]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential  -Check LastGoodCheckDb

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Database.Tests.ps1' with Tags LastGoodCheckDb

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Database.Tests.ps1

  Describing Last Good DBCC CHECKDB

    Context Testing Last Good DBCC CHECKDB on localhost,15592
      [-] last good integrity check for master on 947063dc6ad5 should be less than 7 days old 4ms
        Expected the actual value to be greater than 2020-03-07T17:55:01.3709750Z, because You should have run a DBCC CheckDB inside that time, but got 2019-12-21T14:41:58.0230000.
        200:                                 $psitem.LastGoodCheckDb | Should -BeGreaterThan (Get-Date).ToUniversalTime().AddDays( - ($maxdays)) -Because "You should have run a DBCC CheckDB inside that time"
      

## Check that the Error log has no Errors

By default this will check the error logs for the last 24 hours

In [20]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential  -Check ErrorLog

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Instance.Tests.ps1' with Tags ErrorLog

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Instance.Tests.ps1

  Describing Error Log Entries

    Context Checking error log on localhost,15592
      [+] Error log should be free of error severities 17-24 on localhost,15592 3ms

  Describing Error Log Entries

    Context Checking error log on localhost,15593
      [+] Error log should be free of error severities 17-24 on localhost,15593 2ms
Tests completed in 1.12s
Tests Passed: 2, Failed: 0, Skipped: 0, Pending: 0, Inconclusive: 0 


## Check the disk space on the instance

This will check that the disk space on the hosts has at least 20% free space by default. It requires passing the host names to the `-ComputerName` parameter

In [21]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -ComputerName $SqlInstances -Check DiskCapacity

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Server.Tests.ps1' with Tags DiskCapacity

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Server.Tests.ps1

  Describing Disk Space

    Context Testing Disk Space on localhost,15592
      [+] C:\ with 48.78% free should be at least 20% free on localhost 6ms

  Describing Disk Space

    Context Testing Disk Space on localhost,15593
      [+] C:\ with 48.78% free should be at least 20% free on localhost 8ms
Tests completed in 494ms
Tests Passed: 2, Failed: 0, Skipped: 0, Pending: 0, Inconclusive: 0 


# Check if DAC is enabled

This will check that DAC - the dedicated administrator connection is enabled (by default)

In [22]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential  -Check DAC

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Instance.Tests.ps1' with Tags DAC

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Instance.Tests.ps1

  Describing Dedicated Administrator Connection

    Context Testing Dedicated Administrator Connection on localhost,15592
      [+] DAC is set to True on localhost,15592 97ms

  Describing Dedicated Administrator Connection

    Context Testing Dedicated Administrator Connection on localhost,15593
      [+] DAC is set to True on localhost,15593 101ms
Tests completed in 857ms
Tests Passed: 2, Failed: 0, Skipped: 0, Pending: 0, Inconclusive: 0 


## Check if the database collation matches the server collation

This will check that the collation of the database matches the collation of the instance to avoid collation conflict errors when using temporary tables

In [23]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential  -Check DatabaseCollation

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Database.Tests.ps1' with Tags DatabaseCollation

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Database.Tests.ps1

  Describing Database Collation

    Context Testing database collation on localhost,15592
      [+] database collation (SQL_Latin1_General_CP1_CI_AS) should match server collation (SQL_Latin1_General_CP1_CI_AS) for AdventureWorks2017 on 947063dc6ad5 2ms
      [+] database collation (SQL_Latin1_General_CP1_CI_AS) should match server collation (SQL_Latin1_General_CP1_CI_AS) for master on 947063dc6ad5 2ms
      [+] database collation (SQL_Latin1_General_CP1_CI_AS) should match server collation (SQL_Latin1_General_CP1_CI_AS) for model on 947063dc6ad5 2ms
      [+] database collation (SQ

Are all of the SPNs correct ?

You should not run this one but look at the saved results to see what you can get.

In [6]:
$Instances =  'SQL2005Ser2003', 'SQL2008Ser12R2', 'SQL2012Ser08AG1', 'SQL2012Ser08AG2', 'SQL2012Ser08AG3', 'SQL2014Ser12R2', 'SQL2016N1', 'SQL2016N2', 'SQL2016N3', 'SQL2017N5', 'SQL2019N20', 'SQL2019N21', 'SQL2019N22', 'SQL2019N5'
Invoke-DbcCheck -SqlInstance $instances -Check SPN

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Server.Tests.ps1' with Tags SPN

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Server.Tests.ps1

  Describing SPNs

    Context Testing SPNs on SQL2005Ser2003
      [-] SQL2005Ser2003 should have a SPN Dont know the SPN for Dont know the Account 5ms
        Expected strings to be the same, because We expect to have a SPN Dont know the SPN for Dont know the Account, but they were different.
        Expected length: 4
        Actual length:   17
        Strings differ at index 0.
        Expected: 'None'
        But was:  'An Error occurred'
        191:     $SPN.Error | Should -Be 'None' -Because "We expect to have a SPN $($SPN.RequiredSPN) for $($SPN.InstanceServiceAccount)"
        at Assert-SPN, 

# Running multiple checks

All of the examples above have used a single unique tag. You can run more than one check at a time. You can do this either by specifying a tag that covers a number of checks. You can find these by running `Get-DbcCheck`

When you search for checks like disk

In [7]:
Get-DbcCheck disk

you can see that under AllTags for the DiskCapacity check there is a tag called Storage. If you search for storage

In [8]:
Get-DbcCheck Storage

you can see that this returns two checks - BackUpPathAccess and DiskCapacity. You can run both of these checks at the same time. Note that the `Type` of check is different for these two checks and you will have to supply both `SqlInstance` and `ComputerName`

In [24]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstances -ComputerName $SqlInstances -SqlCredential $SqlCredential -Check Storage

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Instance.Tests.ps1', 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Server.Tests.ps1' with Tags Storage

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Instance.Tests.ps1

  Describing Backup Path Access

    Context Testing Backup Path Access on localhost,15592
      [+] can access backup path (/var/opt/mssql/data) on localhost,15592 105ms

  Describing Backup Path Access

    Context Testing Backup Path Access on localhost,15593
      [+] can access backup path (/var/opt/mssql/data) on localhost,15593 81ms

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Server.Tests.ps1

  Describing Disk Space

    Context Testing Disk Space on localho

## Running checks for a Group

Checks are grouped into ... well groups! The groups are Database, Instance, Server, Agent, HADR, LogShipping and MaintenanceSolution. You can run all of the checks in a group by specifying the group. You will see failures and some errors due to missing configurations and because some checks wont run against containers. I think it is worth showing hte test failures so that you can see what they look like.  

This takes about 90 seconds on my machine

In [25]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential -Check Instance

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Instance.Tests.ps1' with Tags Instance

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.24\checks\Instance.Tests.ps1

  Describing Instance Connection

    Context Testing Instance Connection on localhost,15592
      [+] connects successfully to localhost,15592 5ms
      [-] auth scheme should be Kerberos on localhost,15592 121ms
        Expected strings to be the same, but they were different.
        Expected length: 8
        Actual length:   3
        Strings differ at index 0.
        Expected: 'Kerberos'
        But was:  'SQL'
        64:                         (Test-DbaConnectionAuthScheme -SqlInstance $Instance).authscheme | Should -Be $authscheme
        at <ScriptBlock>, C:\Users\mrrob\Documents\